In [1]:
%load_ext autoreload

In [2]:
import os

import numpy as np

from libertem import api
from libertem.udf.sum import SumUDF
from libertem.udf.base import UDF

In [3]:
%autoreload
ctx = api.Context()

In [4]:
ctx.executor.client.scheduler_info()

{'type': 'Scheduler',
 'id': 'Scheduler-d048e255-8c08-4d2c-a5c2-c9f9aaddf3e3',
 'address': 'tcp://134.94.162.235:44215',
 'services': {},
 'workers': {'tcp://134.94.162.235:33053': {'type': 'Worker',
   'id': 'cpu-1',
   'host': '134.94.162.235',
   'resources': {'CPU': 1, 'compute': 1},
   'local_directory': '/cachedata/users/weber/LiberTEM-uellue/prototypes/cupy/dask-worker-space/worker-bmfeintz',
   'name': 'cpu-1',
   'nthreads': 1,
   'memory_limit': 7504577649,
   'last_seen': 1589313850.4136941,
   'services': {},
   'metrics': {'cpu': 0.0,
    'memory': 27791360,
    'time': 1589313849.0517123,
    'read_bytes': 2972234.795803067,
    'write_bytes': 3574806.314769976,
    'num_fds': 23,
    'executing': 0,
    'in_memory': 0,
    'ready': 0,
    'in_flight': 0,
    'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}}},
   'nanny': 'tcp://134.94.162.235:35196'},
  'tcp://134.94.162.235:33198': {'type': 'Worker',
   'id': 'cpu-16',
   'host': '134.94.162.235',
   'resour

In [5]:
def info():
    import os
    return (os.environ.get("LIBERTEM_USE_CPU", None), os.environ.get("LIBERTEM_USE_CUDA", None))

In [6]:
def backends():
    from libertem.udf.backend import get_backend, get_use_cuda
    return get_backend(), get_use_cuda()

In [7]:
ctx.executor.client.run(backends)

{'tcp://134.94.162.235:33053': ('numpy', None),
 'tcp://134.94.162.235:33198': ('numpy', None),
 'tcp://134.94.162.235:33222': ('numpy', None),
 'tcp://134.94.162.235:33559': ('numpy', None),
 'tcp://134.94.162.235:34485': ('numpy', None),
 'tcp://134.94.162.235:34947': ('numpy', None),
 'tcp://134.94.162.235:36376': ('numpy', None),
 'tcp://134.94.162.235:38303': ('numpy', None),
 'tcp://134.94.162.235:38425': ('numpy', None),
 'tcp://134.94.162.235:38786': ('numpy', None),
 'tcp://134.94.162.235:38996': ('numpy', None),
 'tcp://134.94.162.235:39216': ('numpy', None),
 'tcp://134.94.162.235:41048': ('numpy', None),
 'tcp://134.94.162.235:42505': ('cupy', 0),
 'tcp://134.94.162.235:43519': ('numpy', None),
 'tcp://134.94.162.235:43598': ('numpy', None),
 'tcp://134.94.162.235:44009': ('numpy', None),
 'tcp://134.94.162.235:44544': ('numpy', None),
 'tcp://134.94.162.235:45187': ('numpy', None),
 'tcp://134.94.162.235:45445': ('numpy', None)}

In [8]:
data = np.ones((4, 4, 4, 4))

ds = ctx.load("memory", data=data, num_partitions=4*4)

In [12]:
class DebugUDF(UDF):
    def get_result_buffers(self):
        return {
            'debug': self.buffer(kind="single", dtype="object"),
            'on_device': self.buffer(kind="sig", dtype=np.float32, where="device"),
            'backend': self.buffer(kind="nav", dtype="object"),
        }
    
    def preprocess(self):
        self.results.debug[0] = dict()
    
    def process_partition(self, partition):
        cpu = os.environ.get("LIBERTEM_USE_CPU", None)
        cuda = os.environ.get("LIBERTEM_USE_CUDA", None)
        self.results.debug[0][self.meta.slice] = (cpu, cuda)
        self.results.on_device[:] += self.xp.sum(partition, axis=0)
        self.results.backend[:] = self.meta.backend
        print(f"meta backend {self.meta.backend}")
        
    def merge(self, dest, src):
        de, sr = dest['debug'][0], src['debug'][0]
        for key, value in sr.items():
            assert key not in de
            de[key] = value
        
        dest['on_device'][:] += src['on_device']
        dest['backend'][:] = src['backend']
            
    def get_backends(self):
        return ('cupy', 'numpy')
        

In [13]:
res = ctx.run_udf(udf=DebugUDF(), dataset=ds)
print(res['debug'].data)
print(res['on_device'].data)
print(res['backend'].data)

[{<Slice origin=(6, 0, 0) shape=(1, 4, 4)>: (None, '0'), <Slice origin=(4, 0, 0) shape=(1, 4, 4)>: ('1', None), <Slice origin=(1, 0, 0) shape=(1, 4, 4)>: ('8', None), <Slice origin=(12, 0, 0) shape=(1, 4, 4)>: ('5', None), <Slice origin=(13, 0, 0) shape=(1, 4, 4)>: ('16', None), <Slice origin=(14, 0, 0) shape=(1, 4, 4)>: ('4', None), <Slice origin=(9, 0, 0) shape=(1, 4, 4)>: ('0', None), <Slice origin=(15, 0, 0) shape=(1, 4, 4)>: ('13', None), <Slice origin=(5, 0, 0) shape=(1, 4, 4)>: ('17', None), <Slice origin=(10, 0, 0) shape=(1, 4, 4)>: ('11', None), <Slice origin=(7, 0, 0) shape=(1, 4, 4)>: ('14', None), <Slice origin=(2, 0, 0) shape=(1, 4, 4)>: ('10', None), <Slice origin=(8, 0, 0) shape=(1, 4, 4)>: ('12', None), <Slice origin=(11, 0, 0) shape=(1, 4, 4)>: ('6', None), <Slice origin=(3, 0, 0) shape=(1, 4, 4)>: ('9', None), <Slice origin=(0, 0, 0) shape=(1, 4, 4)>: ('15', None)}]
[[16. 16. 16. 16.]
 [16. 16. 16. 16.]
 [16. 16. 16. 16.]
 [16. 16. 16. 16.]]
[['numpy' 'numpy' 'numpy' 

In [14]:
np.any(res['backend'].data == 'cupy')

True